In [27]:
import win32com.client
import pandas as pd
import numpy as np
import datetime as dt
import time

In [28]:
# 대신증권 API 연결 확인. 1 출력시 연결 성공, 0 출력시 연결 실패
instCpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
print(instCpCybos.IsConnect)

1


In [29]:
# 증권시장에서 상장되어 있는 종목의 갯수 출력 -> 새로운 종목이 상장되거나 폐지됨에 따라 값이 달라진다.
instCpStockCode = win32com.client.Dispatch('CpUtil.CpStockCode')
print(instCpStockCode.GetCount())

3972


## 데이터 수집

In [30]:
# 대신증권 기준으로 정렬된 상장된(주식거래가 가능한) 종목을 전부 가져오기

code = [] # 종목코드
name = [] # 종목명

for i in range(0, instCpStockCode.GetCount()):
    code.append(instCpStockCode.GetData(0, i))
    name.append(instCpStockCode.GetData(1, i))

codeNameList = pd.DataFrame({
      '종목코드':code
    , '종목명':name
})
codeNameList

,종목코드,종목명
0,A000020,동화약품
1,A000040,KR모터스
2,A000050,경방
3,A000070,삼양홀딩스
4,A000075,삼양홀딩스우
...,...,...
3967,J52J720,미래J720호텔신라콜
3968,J52J721,미래J721호텔신라콜
3969,J52J722,미래J722하이브콜
3970,J52J723,미래J723하나금융콜


In [31]:
# 사용자가 원하는 종목의 종목 코드만 데이터프레임으로 출력

jongmokSearchList = ['바이오플러스','HLB','녹십자','제이엘케이'] # 여기에 원하는 종목명 입력
jongmokSearchCode = []
jongmokSearchName = []

for jName in jongmokSearchList:
    jongmokSearchCode.append(codeNameList[codeNameList['종목명'] == jName]['종목코드'].iloc[0])
    jongmokSearchName.append(codeNameList[codeNameList['종목명'] == jName]['종목명'].iloc[0])

jongmokSearchResult = pd.DataFrame({
      '종목코드':jongmokSearchCode
    , '종목명':jongmokSearchName
})
jongmokSearchResult

,종목코드,종목명
0,A099430,바이오플러스
1,A028300,HLB
2,A006280,녹십자
3,A322510,제이엘케이


In [32]:
instStockChart = win32com.client.Dispatch('CpSysDib.CpSvr7254')

In [33]:
Listed = {}
pdListed = []

instStockChart.SetInputValue(0, 'A028300')
instStockChart.SetInputValue(1, 0)
instStockChart.SetInputValue(2, '20210101')
instStockChart.SetInputValue(3, '20230627')
instStockChart.SetInputValue(4, ord('0'))
instStockChart.SetInputValue(5, 0)
instStockChart.SetInputValue(6, ord('1'))

instStockChart.BlockRequest()

num = instStockChart.GetHeaderValue(1)
print('데이터 개수 : '+str(num))

Listed['종목코드'] = jongmokSearchResult['종목코드'].iloc[0]# 종목코드
Listed['종목명'] = jongmokSearchResult['종목명'].iloc[0]    # 종목명
Listed['날짜'] = instStockChart.GetDataValue(0, 11)        # 날짜
Listed['개인'] = instStockChart.GetDataValue(1, 16)        # 개인
Listed['외국인'] = instStockChart.GetDataValue(2, 16)      # 외국인
Listed['기관계'] = instStockChart.GetDataValue(3, 16)      # 기관계
Listed['금융투자'] = instStockChart.GetDataValue(4, 16)    # 금융투자
Listed['보험'] = instStockChart.GetDataValue(5, 16)        # 보험
Listed['투신'] = instStockChart.GetDataValue(6, 16)        # 투신
Listed['은행'] = instStockChart.GetDataValue(7, 16)        # 은행
Listed['기타금융'] = instStockChart.GetDataValue(8, 16)    # 기타금융
Listed['연기금'] = instStockChart.GetDataValue(9, 16)      # 연기금
Listed['기타법인'] = instStockChart.GetDataValue(10, 16)   # 기타법인
Listed['기타외인'] = instStockChart.GetDataValue(11, 16)   # 기타외인
Listed['사모펀드'] = instStockChart.GetDataValue(12, 16)   # 사모펀드
Listed['국가지자체'] = instStockChart.GetDataValue(13, 16) # 국가지자체

pdListed.append(Listed)

dataDf1 = pd.DataFrame(pdListed)

dataDf1

데이터 개수 : 17


,종목코드,종목명,날짜,개인,외국인,기관계,금융투자,보험,투신,은행,기타금융,연기금,기타법인,기타외인,사모펀드,국가지자체
0,A099430,바이오플러스,20230612,922514,170473,-1380373,-1257538,-12668,336215,370,-581,209284,289322,-1948,-655463,0


In [34]:
def getjusik(jusikCode, jusikName, start, end):
    pdListed = []

    instStockChart.SetInputValue(0, jusikCode)
    instStockChart.SetInputValue(1, 0)
    instStockChart.SetInputValue(2, start)
    instStockChart.SetInputValue(3, end)
    instStockChart.SetInputValue(4, ord('0'))
    instStockChart.SetInputValue(5, 0)
    instStockChart.SetInputValue(6, ord('1'))

    instStockChart.BlockRequest()

    num = instStockChart.GetHeaderValue(1)
                                                            
    for i in range(num):
        Listed = {}
        Listed['종목코드'] = jusikCode                             # 종목코드
        Listed['종목명'] = jusikName                               # 종목명
        Listed['날짜'] = instStockChart.GetDataValue(0, i)        # 날짜
        Listed['개인'] = instStockChart.GetDataValue(1, i)        # 개인
        Listed['외국인'] = instStockChart.GetDataValue(2, i)      # 외국인
        Listed['기관계'] = instStockChart.GetDataValue(3, i)      # 기관계
        Listed['금융투자'] = instStockChart.GetDataValue(4, i)    # 금융투자
        Listed['보험'] = instStockChart.GetDataValue(5, i)        # 보험
        Listed['투신'] = instStockChart.GetDataValue(6, i)        # 투신
        Listed['은행'] = instStockChart.GetDataValue(7, i)        # 은행
        Listed['기타금융'] = instStockChart.GetDataValue(8, i)    # 기타금융
        Listed['연기금'] = instStockChart.GetDataValue(9, i)      # 연기금
        Listed['기타법인'] = instStockChart.GetDataValue(10, i)   # 기타법인
        Listed['기타외인'] = instStockChart.GetDataValue(11, i)   # 기타외인
        Listed['사모펀드'] = instStockChart.GetDataValue(12, i)   # 사모펀드

        pdListed.append(Listed)

    dataDf = pd.DataFrame(pdListed)
    return dataDf

In [41]:
a = dt.datetime(2021,1,1)
b = dt.datetime.now()
number = b-a

for j in range(0, jongmokSearchResult.shape[0]):
    frame = []
    for i in range(0, number.days, 2):
        c = a + dt.timedelta(days=i)
        d = a + dt.timedelta(days=i+1)
        result = getjusik(jongmokSearchResult['종목코드'].iloc[j], jongmokSearchResult['종목명'].iloc[j], c.strftime('%Y%m%d'), d.strftime('%Y%m%d'))
        frame.append(result)
        time.sleep(0.3)
    hapdata = pd.concat(frame)
    hapdata.to_csv('./gwajae5/박효민/'+jongmokSearchResult['종목명'].iloc[j]+' 주식 데이터(투자주체별).csv', encoding='euc-kr', index=False)
    print(jongmokSearchResult['종목명'].iloc[j]+' 종목의 투자주체별 데이터 요청 수신성공 및 csv로 저장 성공!')

print('수신요청 종료')

바이오플러스 종목의 투자주체별 데이터 요청 수신성공 및 csv로 저장 성공!
HLB 종목의 투자주체별 데이터 요청 수신성공 및 csv로 저장 성공!
녹십자 종목의 투자주체별 데이터 요청 수신성공 및 csv로 저장 성공!
제이엘케이 종목의 투자주체별 데이터 요청 수신성공 및 csv로 저장 성공!
수신요청 종료
